In [28]:
import pandas as pd

# Merge Dataset

In [29]:
# Load the two DataFrames
df_nutrition = pd.read_csv(r'C:\Intrenship\Torhe Indonesia\Food Nutrition Detection\data\external\all_food_nutritions.csv', delimiter='|')
df_details = pd.read_csv(r'C:\Intrenship\Torhe Indonesia\Food Nutrition Detection\data\interim\all_food_details_cleaned.csv', delimiter='|')

# Merge the two DataFrames
df_merged = pd.merge(df_nutrition, df_details, on='FOOD_CODE', how='inner')

# Save the merged DataFrame to a new CSV file
df_merged.to_csv(r'C:\Intrenship\Torhe Indonesia\Food Nutrition Detection\data\interim\merged_food_data.csv', index=False, sep='|')

# Cleanning Dataset

In [30]:
# Load the dataframe
df_details = pd.read_csv(r"C:\Intrenship\Torhe Indonesia\Food Nutrition Detection\data\external\all_food_details.csv", delimiter='|')

# Drop unused columns
df_details.drop(columns=['Unnamed: 2', 'Latin Name', 'From', 'Description'], inplace=True)

# Save the cleaned dataframe to a new CSV file
df_details.to_csv(r"C:\Intrenship\Torhe Indonesia\Food Nutrition Detection\data\interim\all_food_details_cleaned.csv", index=False, sep='|')

# Data Pivoting

In [31]:
df = pd.read_csv(r'C:\Intrenship\Torhe Indonesia\Food Nutrition Detection\data\interim\merged_food_data.csv', delimiter='|')

# Pivot the DataFrame
df_pivot = df.pivot(index='FOOD_CODE', columns='Nutrient/Ingridient', values='Value (per 100gr)')

# Drop the original columns after pivoting
df = df.drop(columns=['Nutrient/Ingridient', 'Value (per 100gr)'])

# Merge the pivoted DataFrame back to the original DataFrame
df = pd.merge(df, df_pivot, on='FOOD_CODE', how='outer')

# Fill NaN values with 0
df.fillna(0, inplace=True)

# Remove duplicate rows
df.drop_duplicates(inplace=True)

df.to_csv(r'C:\Intrenship\Torhe Indonesia\Food Nutrition Detection\data\interim\pivoted_food_data.csv', index=False, sep='|')

# Data Normalization

In [32]:
df = pd.read_csv(r'C:\Intrenship\Torhe Indonesia\Food Nutrition Detection\data\interim\pivoted_food_data.csv', delimiter='|')

# Clean specific columns by removing non-numeric characters
columns_to_clean = df.columns[4:26]

# Convert columns to string and remove non-numeric characters
for col in columns_to_clean:
    df[col] = df[col].astype(str).str.replace(r'[^\d.]', '', regex=True)


In [33]:
# Add units to the columns
columns_g = ['Abu (ASH)', 'Air (Water)', 'Karbohidrat (CHO)', 'Lemak (Fat)', 'Protein (Protein)', 'Serat (Fibre)', ]
columns_mg = ['Besi (Fe)', 'Fosfor (P)', 'Kalium (K)', 'Kalsium (Ca)', 'Natrium (Na)', 'Niasin (Niacin)', 'Riboflavin (Vit. B2)', 'Seng (Zn)', 'Tembaga (Cu)', 'Thiamin (Vit. B1)', 'Vitamin C (Vit. C)']
columns_mcg = ['Beta-Karoten (Carotenes)', 'Karoten Total (Re)', 'Retinol (Vit. A)']

# Append units to the columns
for col in columns_g:
    df[col] =df[col].astype(str) + ' g'
for col in columns_mg:
    df[col] = df[col].astype(str) + ' mg'
for col in columns_mcg:
    df[col] = df[col].astype(str) + ' mcg'

In [34]:
# Rename columns to lowercase and replace spaces with underscores
df.rename(columns={
    "FOOD_CODE": 'code',
    "Name": 'name',
    "Abu (ASH)": "abu",
    "Air (Water)": "air",
    "Besi (Fe)": "besi",
    "Beta-Karoten (Carotenes)": "beta-karoten",
    "Energi (Energy)": "energi",
    "Fosfor (P)": "fosfor",
    "Kalium (K)": "kalium",
    "Kalsium (Ca)": "kalsium",
    "Karbohidrat (CHO)": "karbohidrat",
    "Karoten Total (Re)": "karoten total",
    "Lemak (Fat)": "lemak",
    "Natrium (Na)": "natrium",
    "Niasin (Niacin)": "niasin",
    "Protein (Protein)": "protein",
    "Retinol (Vit. A)": "retinol",
    "Riboflavin (Vit. B2)": "riboflavin",
    "Seng (Zn)": "seng",
    "Serat (Fibre)": "serat",
    "Tembaga (Cu)": "tembaga",
    "Thiamin (Vit. B1)": "thiamin",
    "Vitamin C (Vit. C)": "vitamin c"
}, inplace=True)

df['name'] = df['name'].str.replace(',', '', regex=False)

df.to_csv(r'C:\Intrenship\Torhe Indonesia\Food Nutrition Detection\data\interim\normalization_data.csv', index=False, sep='|')

# Structured-to-Text Conversion

In [41]:
df = pd.read_csv(r'C:\Intrenship\Torhe Indonesia\Food Nutrition Detection\data\interim\normalization_data.csv', delimiter='|')

# Convert the 'Name' column to string
name_nutrition = list(df.columns[4:])

# Create a new 'Text' column with the desired format
df['Text'] = df['name'] + ' memiliki kandungan zat gizi seperti '

for i, col in enumerate(name_nutrition):
    if i == len(name_nutrition) - 1:
        separator = '. '
    elif i == len(name_nutrition) - 2:
        separator = ' dan '
    else:
        separator = ', '
    df['Text'] += df[col].astype(str) + f' zat {col}' + separator

# Remove unused columns
df = df[['code', 'name', 'Text']]

df.to_csv(r'C:\Intrenship\Torhe Indonesia\Food Nutrition Detection\data\processed\Food_Dataset.csv', index=False, sep='|')